In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from ares_py.get_ld import get_ld
from ares_py.class_ert import ERT
from ares_py.plot.fig import fig_meas_data
from ares_py.geometry.coords import (
    coords_load,
    coords_ld2d,
    coords_interpolate,
    coord_merge,
    coords_merge_sections,
)

fps = get_ld("input", ext=".2dm")["fp"]

for fp in fps:
    print(fp)
    ert = ERT(fp).Load().Colorscales()

    try:
        coords = coords_load(ert.fp_load.replace(".2dm", ".csv"))
        coords = coords_ld2d(coords)

        ert.coords_int = coords_interpolate(coords)
        ert.data = coord_merge(ert)
        ert.sec = coords_merge_sections(ert)
    except:
        print("\tError reading topo data..")
    # fig = fig_meas_data(ert)


ert.data

In [ ]:
from ares_py.geometry.dtm import tif_read

dtm = tif_read("input/klinovec/dtm.tif")

In [ ]:
import geopandas as gpd
import shapely
from shapely import LineString


def dtm_get_buffer(ert):
    data = ert.sec.copy().dropna(subset="x")
    geom = gpd.points_from_xy(data["x"], data["y"])

    geom = LineString(geom)
    buffer = shapely.buffer(geom, 5)
    return buffer


def dtm_clip_line_lim(ert, dtm):
    df = pd.DataFrame(np.column_stack([d.flatten() for d in dtm]))

    mask2 = df[0] <= ert.sec["x"].max()
    mask1 = df[0] >= ert.sec["x"].min()
    df = df.loc[mask1 & mask2]

    mask2 = df[1] <= ert.sec["y"].max()
    mask1 = df[1] >= ert.sec["y"].min()
    df = df.loc[mask1 & mask2]
    df = df.dropna()
    return df


def dtm_clip_buffer():
    buffer = dtm_get_buffer(ert)
    df = dtm_clip_line_lim(ert, dtm)

    pt = gpd.GeoSeries(gpd.points_from_xy(df[0], df[1], df[2]))
    mask = buffer.contains(pt)

    dtm_clipped = pt[mask == True]
    dtm_clipped = dtm_clipped.get_coordinates(include_z=True)
    return

[array([[-839397.01944944, -839397.01944944, -839397.01944944, ...,
         -839397.01944944, -839397.01944944, -839397.01944944],
        [-839396.01834831, -839396.01834831, -839396.01834831, ...,
         -839396.01834831, -839396.01834831, -839396.01834831],
        [-839395.01724719, -839395.01724719, -839395.01724719, ...,
         -839395.01724719, -839395.01724719, -839395.01724719],
        ...,
        [-838803.36648315, -838803.36648315, -838803.36648315, ...,
         -838803.36648315, -838803.36648315, -838803.36648315],
        [-838802.36538202, -838802.36538202, -838802.36538202, ...,
         -838802.36538202, -838802.36538202, -838802.36538202],
        [-838801.3642809 , -838801.3642809 , -838801.3642809 , ...,
         -838801.3642809 , -838801.3642809 , -838801.3642809 ]],
       shape=(596, 445)),
 array([[-993408.38423272, -993409.38349815, -993410.38276359, ...,
         -993850.05955554, -993851.05882097, -993852.05808641],
        [-993408.38423272, -993409.3